<a href="https://colab.research.google.com/github/wung-satalo/SearchEngine/blob/main/IR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
pip install requests

In [11]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

In [12]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
url="https://sportsteamhistory.com/premier-team-history/"
response = requests.get(url)

In [14]:
response

<Response [200]>

In [15]:
page_html = BeautifulSoup(response.text, 'html.parser')
page_html

<!DOCTYPE html>

<html class="no-js" dir="ltr" lang="en-US" prefix="og: https://ogp.me/ns#">
<head><meta charset="utf-8"/><script>if(navigator.userAgent.match(/MSIE|Internet Explorer/i)||navigator.userAgent.match(/Trident\/7\..*?rv:11/i)){var href=document.location.href;if(!href.match(/[?&]nowprocket/)){if(href.indexOf("?")==-1){if(href.indexOf("#")==-1){document.location.href=href+"?nowprocket=1"}else{document.location.href=href.replace("#","?nowprocket=1#")}}else{if(href.indexOf("#")==-1){document.location.href=href+"&nowprocket=1"}else{document.location.href=href.replace("#","&nowprocket=1#")}}}}</script><script>(()=>{class RocketLazyLoadScripts{constructor(){this.v="2.0.3",this.userEvents=["keydown","keyup","mousedown","mouseup","mousemove","mouseover","mouseenter","mouseout","mouseleave","touchmove","touchstart","touchend","touchcancel","wheel","click","dblclick","input","visibilitychange"],this.attributeEvents=["onblur","onclick","oncontextmenu","ondblclick","onfocus","onmousedow

In [16]:
pre_containers = page_html.find('div',{"class":"entry-content content"})
pre_containers

<div class="entry-content content">
<div class="addtoany_share_save_container addtoany_content addtoany_content_top"><div class="a2a_kit a2a_kit_size_22 addtoany_list" data-a2a-title="Premier Team History" data-a2a-url="https://sportsteamhistory.com/premier-team-history/"><a class="a2a_button_facebook" href="https://www.addtoany.com/add_to/facebook?linkurl=https%3A%2F%2Fsportsteamhistory.com%2Fpremier-team-history%2F&amp;linkname=Premier%20Team%20History" rel="nofollow noopener" target="_blank" title="Facebook"></a><a class="a2a_button_twitter" href="https://www.addtoany.com/add_to/twitter?linkurl=https%3A%2F%2Fsportsteamhistory.com%2Fpremier-team-history%2F&amp;linkname=Premier%20Team%20History" rel="nofollow noopener" target="_blank" title="Twitter"></a><a class="a2a_button_linkedin" href="https://www.addtoany.com/add_to/linkedin?linkurl=https%3A%2F%2Fsportsteamhistory.com%2Fpremier-team-history%2F&amp;linkname=Premier%20Team%20History" rel="nofollow noopener" target="_blank" title="

In [17]:
overview_link = []
Club = []
for link in pre_containers.find_all('a',{'tabindex':'0'}):
  prelink = link['href']
  overview_link.append(prelink)

for name in pre_containers.find_all('h4',{'style':'text-align: left;'}):
  clubname = name.text
  Club.append(clubname)

print(Club)
print(overview_link)

['AFC Bournemouth', 'Arsenal FC', 'Aston Villa FC', 'Brentford FC', 'Brighton & Hove Albion FC', 'Chelsea FC', 'Crystal Palace FC', 'Everton FC', 'Fulham FC', 'Leicester City FC', 'Liverpool FC', 'Manchester City FC', 'Manchester United FC', 'Newcastle United FC', 'Nottingham Forest FC', 'Southhampton FC', 'Tottenham Hotspur FC', 'West Ham United FC', 'Wolverhampton Wanderers FC']
['/afc-bournemouth', '/arsenal-fc', '/aston-villa-fc', '/brentford-fc', '/brighton-hove-albion-fc', '/chelsea-fc', '/crystal-palace-fc', '/everton-fc', '/fulham-fc', '/leicester-city-fc/', '/liverpool-fc', '/manchester-city-fc', '/manchester-united-fc', '/newcastle-united-fc', '/nottingham-forest-fc', '/southampton-fc/', '/tottenham-hotspur-fc', '/west-ham-united-fc', '/wolverhampton-wanderers-fc']


In [18]:
root = 'https://sportsteamhistory.com/'
History = []

for linkPage in overview_link:
  result_each_club = requests.get(f'{root}{linkPage}')
  soup_each_club = BeautifulSoup(result_each_club.content, 'html.parser')
  history = soup_each_club.find_all('div',{'x-text'})[2]
  his = history.text
  History.append(his)





In [19]:
print(History[0])



Who is the greatest AFC Bournemouth?


 Doug McGibbon
						



 Callum Wilson
						



 Tony Powell*
						



 Dickie Dowsett
						



 Stan Newsham
						



 Ron Eyre
						



 James Hayter
						



 Ted MacDougall
						



 Brett Pitman
						



 Steve Fletcher
						



 Jack Cross
						
Add your greatest playerPoll Options are limited because JavaScript is disabled in your browser.To qualify as the greatest player for this team, the player must have played one season for this team. If not, we will remove the player.
* verifies that player has played for this team as an added player by a fan.















In [26]:
lemma = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

ClubRege = list(range(len(Club)))
HistoryRege = list(range(len(History)))

min_len = min(len(Club), len(History))

for i in range(0,min_len):
    tokenizer = nltk.RegexpTokenizer("[\w']+")

    regetokened_data = []
    text_club = Club[i]
    lemma_words = lemma.lemmatize(text_club,pos='v')
    regetokened_data = tokenizer.tokenize(lemma_words)
    filtered_words = []
    filtered_words = [club for club in regetokened_data if not club.lower() in stop_words]
    ClubRege[i] = filtered_words

    regetokened_data = []
    text_sta = History[i]
    lemma_words = lemma.lemmatize(text_sta,pos='v')
    regetokened_data = tokenizer.tokenize(lemma_words)
    filtered_words = []
    filtered_words = [sta for sta in regetokened_data if not sta.lower() in stop_words]
    HistoryRege[i] = regetokened_data

print(ClubRege)
print(HistoryRege)

[['AFC', 'Bournemouth'], ['Arsenal', 'FC'], ['Aston', 'Villa', 'FC'], ['Brentford', 'FC'], ['Brighton', 'Hove', 'Albion', 'FC'], ['Chelsea', 'FC'], ['Crystal', 'Palace', 'FC'], ['Everton', 'FC'], ['Fulham', 'FC'], ['Leicester', 'City', 'FC'], ['Liverpool', 'FC'], ['Manchester', 'City', 'FC'], ['Manchester', 'United', 'FC'], ['Newcastle', 'United', 'FC'], ['Nottingham', 'Forest', 'FC'], ['Southhampton', 'FC'], ['Tottenham', 'Hotspur', 'FC'], ['West', 'Ham', 'United', 'FC'], ['Wolverhampton', 'Wanderers', 'FC']]
[['Who', 'is', 'the', 'greatest', 'AFC', 'Bournemouth', 'Doug', 'McGibbon', 'Callum', 'Wilson', 'Tony', 'Powell', 'Dickie', 'Dowsett', 'Stan', 'Newsham', 'Ron', 'Eyre', 'James', 'Hayter', 'Ted', 'MacDougall', 'Brett', 'Pitman', 'Steve', 'Fletcher', 'Jack', 'Cross', 'Add', 'your', 'greatest', 'playerPoll', 'Options', 'are', 'limited', 'because', 'JavaScript', 'is', 'disabled', 'in', 'your', 'browser', 'To', 'qualify', 'as', 'the', 'greatest', 'player', 'for', 'this', 'team', 'the'

In [27]:
Data_df = pd.DataFrame({
    'Club':ClubRege,
    'History':HistoryRege
})
print(Data_df)


                              Club  \
0               [AFC, Bournemouth]   
1                    [Arsenal, FC]   
2               [Aston, Villa, FC]   
3                  [Brentford, FC]   
4     [Brighton, Hove, Albion, FC]   
5                    [Chelsea, FC]   
6            [Crystal, Palace, FC]   
7                    [Everton, FC]   
8                     [Fulham, FC]   
9            [Leicester, City, FC]   
10                 [Liverpool, FC]   
11          [Manchester, City, FC]   
12        [Manchester, United, FC]   
13         [Newcastle, United, FC]   
14        [Nottingham, Forest, FC]   
15              [Southhampton, FC]   
16        [Tottenham, Hotspur, FC]   
17         [West, Ham, United, FC]   
18  [Wolverhampton, Wanderers, FC]   

                                              History  
0   [Who, is, the, greatest, AFC, Bournemouth, Dou...  
1   [Who, is, the, greatest, Arsenal, FC, Robin, v...  
2   [Who, is, the, greatest, Aston, Villa, FC, Dwi...  
3   [Who, is, t

In [28]:
Data_df.to_csv('PremierLeague.csv')

In [29]:
df = pd.read_csv('PremierLeague.csv')
df

,Unnamed: 0,Club,History
0,0,"['AFC', 'Bournemouth']","['Who', 'is', 'the', 'greatest', 'AFC', 'Bourn..."
1,1,"['Arsenal', 'FC']","['Who', 'is', 'the', 'greatest', 'Arsenal', 'F..."
2,2,"['Aston', 'Villa', 'FC']","['Who', 'is', 'the', 'greatest', 'Aston', 'Vil..."
3,3,"['Brentford', 'FC']","['Who', 'is', 'the', 'greatest', 'Brentford', ..."
4,4,"['Brighton', 'Hove', 'Albion', 'FC']","['Who', 'is', 'the', 'greatest', 'Brighton', '..."
5,5,"['Chelsea', 'FC']","['Who', 'is', 'the', 'greatest', 'Chelsea', 'F..."
6,6,"['Crystal', 'Palace', 'FC']","['Who', 'is', 'the', 'greatest', 'Crystal', 'P..."
7,7,"['Everton', 'FC']","['Who', 'is', 'the', 'greatest', 'Everton', 'F..."
8,8,"['Fulham', 'FC']","['Who', 'is', 'the', 'greatest', 'Fulham', 'FC..."
9,9,"['Leicester', 'City', 'FC']","['Who', 'is', 'the', 'greatest', 'Leicester', ..."


In [30]:
from ast import literal_eval

In [31]:

data={}
count=0
for index, row in df.iterrows():

  Data =[]
  Data.extend(eval(row[1]))
  Data.extend(eval(row[2]))

  data[int(row[0])]=Data

  print(data[int(row[0])])


['AFC', 'Bournemouth', 'Who', 'is', 'the', 'greatest', 'AFC', 'Bournemouth', 'Doug', 'McGibbon', 'Callum', 'Wilson', 'Tony', 'Powell', 'Dickie', 'Dowsett', 'Stan', 'Newsham', 'Ron', 'Eyre', 'James', 'Hayter', 'Ted', 'MacDougall', 'Brett', 'Pitman', 'Steve', 'Fletcher', 'Jack', 'Cross', 'Add', 'your', 'greatest', 'playerPoll', 'Options', 'are', 'limited', 'because', 'JavaScript', 'is', 'disabled', 'in', 'your', 'browser', 'To', 'qualify', 'as', 'the', 'greatest', 'player', 'for', 'this', 'team', 'the', 'player', 'must', 'have', 'played', 'one', 'season', 'for', 'this', 'team', 'If', 'not', 'we', 'will', 'remove', 'the', 'player', 'verifies', 'that', 'player', 'has', 'played', 'for', 'this', 'team', 'as', 'an', 'added', 'player', 'by', 'a', 'fan']
['Arsenal', 'FC', 'Who', 'is', 'the', 'greatest', 'Arsenal', 'FC', 'Robin', 'van', 'Persie', 'Walley', 'Barnes', 'Mesut', 'Özil', 'Dennis', 'Bergkamp', 'Olivier', 'Giroud', 'Cesc', 'Fàbregas', 'Fredrik', 'Ljungberg', 'Thierry', 'Henry', 'Robert

<ipython-input-31-52b390a3c09d>:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  Data.extend(eval(row[1]))
<ipython-input-31-52b390a3c09d>:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  Data.extend(eval(row[2]))
<ipython-input-31-52b390a3c09d>:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  data[int(row[0])]=Data
<ipython-input-31-52b390a3c09d>:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a f

In [32]:
data_list=[]

for i in data:

  my_string = ','.join(data[i])

  data_list.append(my_string)
  print(my_string)

AFC,Bournemouth,Who,is,the,greatest,AFC,Bournemouth,Doug,McGibbon,Callum,Wilson,Tony,Powell,Dickie,Dowsett,Stan,Newsham,Ron,Eyre,James,Hayter,Ted,MacDougall,Brett,Pitman,Steve,Fletcher,Jack,Cross,Add,your,greatest,playerPoll,Options,are,limited,because,JavaScript,is,disabled,in,your,browser,To,qualify,as,the,greatest,player,for,this,team,the,player,must,have,played,one,season,for,this,team,If,not,we,will,remove,the,player,verifies,that,player,has,played,for,this,team,as,an,added,player,by,a,fan
Arsenal,FC,Who,is,the,greatest,Arsenal,FC,Robin,van,Persie,Walley,Barnes,Mesut,Özil,Dennis,Bergkamp,Olivier,Giroud,Cesc,Fàbregas,Fredrik,Ljungberg,Thierry,Henry,Robert,Pirès,Ashley,Cole,Theo,Walcott,Add,your,greatest,playerPoll,Options,are,limited,because,JavaScript,is,disabled,in,your,browser,To,qualify,as,the,greatest,player,for,this,team,the,player,must,have,played,one,season,for,this,team,If,not,we,will,remove,the,player,verifies,that,player,has,played,for,this,team,as,an,added,player,by,a,f

In [33]:
data2=data_list

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
doc = vectorizer.fit_transform(data_list)
vectorizer.get_feature_names_out()

array(['aas', 'adcock', 'add', 'added', 'adebayor', 'afc', 'agüero',
       'alan', 'albion', 'alonso', 'an', 'anders', 'anderson', 'anderton',
       'andy', 'antonio', 'archie', 'are', 'arjen', 'arnold', 'arsenal',
       'arthur', 'as', 'ashley', 'aston', 'bale', 'banks', 'barnes',
       'beardsley', 'because', 'beckham', 'bedford', 'benrahma', 'beram',
       'bergkamp', 'berry', 'bert', 'bill', 'billy', 'blanchflower',
       'bobby', 'bournemouth', 'branislav', 'brentford', 'brett', 'brian',
       'brighton', 'broadbent', 'browser', 'bruyne', 'bryn', 'bull',
       'burns', 'by', 'byrne', 'callum', 'cameron', 'campbell', 'cancelo',
       'canio', 'canos', 'carlos', 'carloz', 'carragher', 'carroll',
       'case', 'cesc', 'chamberlain', 'chandler', 'channon', 'chelsea',
       'city', 'clough', 'cody', 'cole', 'coleman', 'colin', 'coloccini',
       'connor', 'conway', 'costa', 'coutinho', 'cristiano', 'cross',
       'crystal', 'dalglish', 'dalrymple', 'danny', 'darren', 'darw

In [35]:
doc_1=doc
df1 = pd.DataFrame(doc_1.toarray(), columns=vectorizer.get_feature_names_out())
df1

,aas,adcock,add,added,adebayor,afc,agüero,alan,albion,alonso,...,winston,wolverhampton,wright,xabi,yaya,yorke,your,zlatan,özil,čech
0,0.000000,0.000000,0.047314,0.047314,0.000000,0.312519,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.094629,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.050061,0.050061,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100121,0.000000,0.165329,0.000000
2,0.000000,0.000000,0.050772,0.050772,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.147093,0.101544,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.054615,0.054615,0.000000,0.000000,0.000000,0.142515,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.109231,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.048123,0.048123,0.000000,0.000000,0.000000,0.000000,0.317862,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.096246,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.047392,0.047392,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.094785,0.000000,0.000000,0.156517
6,0.000000,0.000000,0.059665,0.059665,0.000000,0.000000,0.000000,0.155693,0.000000,0.000000,...,0.000000,0.000000,0.172858,0.000000,0.000000,0.000000,0.119331,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.047440,0.047440,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.094880,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.040782,0.040782,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.081563,0.000000,0.000000,0.000000
9,0.000000,0.170448,0.051611,0.051611,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.103221,0.000000,0.000000,0.000000


In [36]:
stopword=set(stopwords.words('english'))

In [37]:
def get_token(doc_text):
  tokenizer = nltk.RegexpTokenizer("[\w']+")
  tokens = tokenizer.tokenize(doc_text)
  return tokens

In [38]:
def remove_stopword(doc_text):
  cleaned_text=[]
  for words in doc_text:
    if words not in stopword:
      cleaned_text.append(words)
  return cleaned_text

In [39]:
def get_lemma(token_list):
  lem = nltk.stem.WordNetLemmatizer()
  lemma=[]
  for words in token_list:
    lemma.append(lem.lemmatize(words))
  return lemma

In [40]:
query =input(' ')

query=get_token(query)
query=remove_stopword(query)
#print(query)
q=[]
for word in get_lemma(query):
  q.append(word)
q=' '.join(q)
print(q)

 Arsenal
Arsenal


In [41]:
query_1 = vectorizer.transform([q])
df1 = pd.DataFrame(query_1.toarray(), columns=vectorizer.get_feature_names_out())
df1

,aas,adcock,add,added,adebayor,afc,agüero,alan,albion,alonso,...,winston,wolverhampton,wright,xabi,yaya,yorke,your,zlatan,özil,čech
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(doc_1,query_1).flatten()

a = cosine_sim.argsort()[:-11:-1]
print(a)

[ 1 17 18 16 15 13 14 11 10  9]


In [43]:
for i in a:
  data = [data2[i]]
  if(cosine_sim[i]==0.0):
    break
  print('simialriy = '+str(cosine_sim[i]))
  print(data,"\n")

simialriy = 0.33065871448301176
['Arsenal,FC,Who,is,the,greatest,Arsenal,FC,Robin,van,Persie,Walley,Barnes,Mesut,Özil,Dennis,Bergkamp,Olivier,Giroud,Cesc,Fàbregas,Fredrik,Ljungberg,Thierry,Henry,Robert,Pirès,Ashley,Cole,Theo,Walcott,Add,your,greatest,playerPoll,Options,are,limited,because,JavaScript,is,disabled,in,your,browser,To,qualify,as,the,greatest,player,for,this,team,the,player,must,have,played,one,season,for,this,team,If,not,we,will,remove,the,player,verifies,that,player,has,played,for,this,team,as,an,added,player,by,a,fan'] 

